<a href="https://colab.research.google.com/github/inamansari21/datascience/blob/main/MLR_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from numpy.polynomial.polynomial import polyfit
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.stats.tests.test_influence
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
df = pd.read_csv("ToyotaCorolla.csv",encoding='latin1')


In [ ]:
df1 = df [['Price','Age_08_04','KM','HP','cc','Doors','Gears','Quarterly_Tax','Weight']]
df1

In [ ]:
df1.describe


In [ ]:
sns.pairplot(df1)


# **correlation matrix**

In [ ]:
corrMatrix = df1.corr()


In [ ]:
sns.heatmap(corrMatrix, annot=True)


# **regression model**

In [ ]:
X = df1[['Age_08_04','KM','HP','cc','Doors','Gears','Quarterly_Tax','Weight']]
Y = df1[['Price']]


In [ ]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X)
           

In [ ]:
model.summary()

**R sq and p Value of the Model is Good and the model can be accepted. However as you can see not all variables have acceptable p value. Thus we have Multicollinearity issue in our Data Frame
**

# **solving multicollinearity**
# **finding cook's distance**

In [ ]:
infl = model.get_influence()


In [ ]:
summ_df = infl.summary_frame()


In [ ]:
summ_df.sort_values('cooks_d', ascending=False)


In [ ]:
infl.plot_influence()


Index 80 has highest Cook's Distance
Thus it will be wise to drop this row from the data set

bold text

# **finding VIF**

In [ ]:
vif = pd.DataFrame()


In [ ]:
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]



In [ ]:
vif["features"] = X.columns


In [ ]:
vif.round(1)


**As expected, Geras and Weight have a high variance inflation factor because they "explain" the same variance within this dataset. We would need to discard one of these variables to improve model and try to solve multicolinearity.
**

In [ ]:
# Removed Weight fron the dataframe and Tested the model
new_X = df1[['Age_08_04','KM','HP','cc','Doors','Gears','Quarterly_Tax']]


In [ ]:
new_model = sm.OLS(Y, new_X).fit()
new_predictions = new_model.predict(new_X)


In [ ]:
new_model.summary()


 once we remove "Weight" from input variables and run the model again, all the variables are significant.


# **final model**

In [ ]:
# Removed Index with highest Cook's distance to remove the hightest influencial ROW
new_df = df1.drop(df1.index[80])


In [ ]:
new_X = new_df[['Age_08_04','KM','HP','cc','Doors','Gears','Quarterly_Tax']]
new_Y = new_df[['Price']]


In [ ]:
final_model = sm.OLS(new_Y, new_X).fit()
predictions = final_model.predict(new_X)


In [ ]:
final_model.summary()


# **output interpretation**
1 - p-value < 0.01
Thus the model is accepted

2 - coefficient == -132.01
Thus if the value of x increased by 1, the predicted value of Price will decrease by 132.01

3 - Adj. R-sqared == 0.983
Thus the model explains 98.3% of the variance in dependent variable

